In [1]:
#imports
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import cv2
import os
from IPython.display import Image, display
import shutil
import joblib
from keras.models import load_model



C:\Users\Jonah\miniconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [8]:
#Define constants used throughout
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_SIZE=(128,128)
IMG_CHANNELS=3
MODEL_NAME = "CatsOrDogsModelSlim.keras"


In [9]:
#Normalize and prepare our training data
Directory = os.listdir("./trainSet/train")


dataLabels=[]
for name in Directory:
    label = name.split('.')[0]
    if label == 'dog':
        dataLabels.append(1)
    else:
        dataLabels.append(0)

df = pd.DataFrame({
    'filename':Directory,
    'label':dataLabels
})
# Log the data frame to ensure we are labeling our training set correctly
print(df)

           filename  label
0         cat.0.jpg      0
1         cat.1.jpg      0
2        cat.10.jpg      0
3       cat.100.jpg      0
4      cat.1000.jpg      0
...             ...    ...
24995  dog.9995.jpg      1
24996  dog.9996.jpg      1
24997  dog.9997.jpg      1
24998  dog.9998.jpg      1
24999  dog.9999.jpg      1

[25000 rows x 2 columns]


In [10]:
# Model definition. Containing three feature extraction layers
# as well as a Classification fully connected layer.
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dropout,
    Flatten, Dense, BatchNormalization
)

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = Conv2D(64, (3,3), activation='relu')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(2,2)(x)
x = Dropout(0.25)(x)


x = Conv2D(64, (3,3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2,2)(x)
x = Dropout(0.25)(x)

#Name the last convolutional layer, so that we can grab the output for our GRAD-CAM
x = Conv2D(128, (3,3), activation='relu', name="last_conv")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2,2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

outputs = Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
#log a summary of the model.
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 126, 126, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 126, 126, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 61, 61, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ last_conv (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,962,242 (49.45 MB)

 Trainable params: 12,960,706 (49.44 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learningRateReduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001,
                                            mode='max')
callbacks = [earlystop, learningRateReduction]

In [12]:
df["label"] = df["label"].replace({0: 'cat', 1: 'dog'}) 

In [13]:
trainImgs, validationImgs = train_test_split(df, test_size=0.20, random_state=42)
trainImgs = trainImgs.reset_index(drop=True)
validationImgs = validationImgs.reset_index(drop=True)


In [14]:
encodedTrainData = trainImgs.shape[0]
encodedValidationData = validationImgs.shape[0]
batch_size=15

In [15]:
trainDataGen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

trainData = trainDataGen.flow_from_dataframe(trainImgs,
                                             "./trainSet/train/",x_col='filename',y_col='label',
                                                 target_size=IMG_SIZE,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)


Found 20000 validated image filenames belonging to 2 classes.


In [16]:
validateDataGen = ImageDataGenerator(rescale=1./255)
validationData = validateDataGen.flow_from_dataframe(
    validationImgs, 
    "./trainSet/train/", 
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)


Found 5000 validated image filenames belonging to 2 classes.


In [17]:
epochs=10
history = model.fit(
    trainData, 
    epochs=epochs,
    validation_data=validationData,
    validation_steps=encodedValidationData//batch_size,
    steps_per_epoch=encodedTrainData//batch_size,
    callbacks=callbacks
)

Epoch 1/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 132s 98ms/step - accuracy: 0.6176 - loss: 0.7488 - val_accuracy: 0.6529 - val_loss: 0.7055 - learning_rate: 0.0010
Epoch 2/10
   1/1333 ━━━━━━━━━━━━━━━━━━━━ 1:51 83ms/step - accuracy: 0.4000 - loss: 0.8295

C:\Users\Jonah\miniconda3\Lib\site-packages\keras\src\callbacks\callback_list.py:171: UserWarning: Learning rate reduction is conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
C:\Users\Jonah\miniconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1333/1333 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4000 - loss: 0.8295 - val_accuracy: 0.6348 - val_loss: 0.7244 - learning_rate: 0.0010
Epoch 3/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 126s 95ms/step - accuracy: 0.7024 - loss: 0.5822 - val_accuracy: 0.7678 - val_loss: 0.4966 - learning_rate: 0.0010
Epoch 4/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8000 - loss: 0.4700 - val_accuracy: 0.7644 - val_loss: 0.5002 - learning_rate: 0.0010
Epoch 5/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 126s 95ms/step - accuracy: 0.7479 - loss: 0.5218 - val_accuracy: 0.6450 - val_loss: 0.7628 - learning_rate: 0.0010
Epoch 6/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7333 - loss: 0.5803 - val_accuracy: 0.6260 - val_loss: 0.8193 - learning_rate: 0.0010
Epoch 7/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 126s 94ms/step - accuracy: 0.7839 - loss: 0.4717 - val_accuracy: 0.7972 - val_loss: 0.4329 - learning_rate: 0.0010
Epoch 8/10
1333/1333 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6667 - los

In [18]:
model.save(MODEL_NAME)

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter.ttk import Progressbar, Style
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import cv2
import matplotlib as mpl
from keras.models import load_model

# =========================
# CONFIG
# =========================
LAST_CONV_LAYER = "last_conv"

CLASSES = {
    0: "Image is of a Cat 🐱",
    1: "Image is of a Dog 🐶"
}

# UI COLORS
BG_COLOR = "#0f172a"
CARD_COLOR = "#111827"
ACCENT = "#38bdf8"
TEXT = "#e5e7eb"
MUTED = "#94a3b8"

# =========================
# LOAD MODEL
# =========================
model = load_model(MODEL_NAME)

# =========================
# IMAGE HELPERS
# =========================
def get_img_array(img_path, size):
    img = tf.keras.utils.load_img(img_path, target_size=size)
    array = tf.keras.utils.img_to_array(img) / 255.0
    return np.expand_dims(array, axis=0)

# =========================
# GRAD-CAM
# =========================
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        model.inputs,
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_output = conv_output[0]
    heatmap = conv_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0)
    heatmap = tf.pow(heatmap, 0.5)
    heatmap /= tf.reduce_max(heatmap) + 1e-8

    return heatmap.numpy()

def save_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.6):
    img = tf.keras.utils.load_img(img_path)
    img = tf.keras.utils.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)
    jet = mpl.colormaps["jet"]
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

    superimposed = jet_heatmap * alpha + img
    superimposed = tf.keras.utils.array_to_img(superimposed)
    superimposed.save(cam_path)

# =========================
# PREDICTION PIPELINE
# =========================
def predict_and_generate_cam(img_path):
    img_array = get_img_array(img_path, IMG_SIZE)

    preds = model.predict(img_array)
    pred_index = np.argmax(preds[0])
    confidence = float(preds[0][pred_index])

    heatmap = make_gradcam_heatmap(
        img_array, model, LAST_CONV_LAYER, pred_index
    )
    save_gradcam(img_path, heatmap)

    return CLASSES[pred_index], confidence, "cam.jpg"

# =========================
# TKINTER UI
# =========================
top = tk.Tk()
top.title("Cat vs Dog Classifier")
top.geometry("1000x800")
top.configure(bg=BG_COLOR)

# HEADER
heading = tk.Label(
    top,
    text="🐱 Cat vs Dog Classifier 🐶",
    font=("Segoe UI", 28, "bold"),
    bg=BG_COLOR,
    fg=ACCENT
)
heading.pack(pady=(20, 10))

# IMAGE CARD
image_card = tk.Frame(
    top,
    bg=CARD_COLOR,
    highlightthickness=1,
    highlightbackground="#1f2933"
)
image_card.pack(pady=20)

image_label = tk.Label(image_card, bg=CARD_COLOR)
image_label.pack(padx=20, pady=20)

# PREDICTION LABEL
prediction_label = tk.Label(
    top,
    text="Upload an image to begin",
    font=("Segoe UI", 20, "bold"),
    bg=BG_COLOR,
    fg=TEXT
)
prediction_label.pack(pady=10)

# CONFIDENCE BAR
style = Style()
style.theme_use("default")
style.configure(
    "Confidence.Horizontal.TProgressbar",
    troughcolor=CARD_COLOR,
    background=ACCENT,
    thickness=20
)

confidence_bar = Progressbar(
    top,
    style="Confidence.Horizontal.TProgressbar",
    orient="horizontal",
    length=400,
    mode="determinate"
)
confidence_bar.pack(pady=10)

# STATUS
status_label = tk.Label(
    top,
    text="Ready",
    font=("Segoe UI", 10),
    bg=BG_COLOR,
    fg=MUTED
)
status_label.pack(pady=5)

# =========================
# BUTTON ACTION
# =========================
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_curve, auc
)
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

def evaluate_model():
    # Load images from organized folder
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

    generator = datagen.flow_from_directory(
        directory="./testSet/organized",
        target_size=IMG_SIZE,
        batch_size=32,
        class_mode="binary",
        shuffle=False
    )

    # True labels
    y_true = generator.classes

    # Predictions
    steps = int(np.ceil(generator.samples / generator.batch_size))
    y_probs = model.predict(generator, steps=steps)
    y_pred = (y_probs[:, 1] > 0.5).astype(int)
    confidences = np.max(y_probs, axis=1)

    # Metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # ROC
    fpr, tpr, _ = roc_curve(y_true, y_probs[:, 1])
    roc_auc = auc(fpr, tpr)

    # Create dashboard window
    dash = tk.Toplevel()
    dash.title("Model Evaluation Dashboard")
    dash.geometry("1000x800")
    dash.configure(bg=BG_COLOR)

    # Display metrics
    tk.Label(dash, text=f"Accuracy: {acc:.3f}", font=("Segoe UI", 18), bg=BG_COLOR, fg=TEXT).pack(pady=10)
    tk.Label(dash, text=f"Precision: {prec:.3f}", font=("Segoe UI", 18), bg=BG_COLOR, fg=TEXT).pack(pady=10)
    tk.Label(dash, text=f"Recall: {rec:.3f}", font=("Segoe UI", 18), bg=BG_COLOR, fg=TEXT).pack(pady=10)
    tk.Label(dash, text=f"F1 Score: {f1:.3f}", font=("Segoe UI", 18), bg=BG_COLOR, fg=TEXT).pack(pady=10)

    # Plot confusion matrix
    fig1, ax1 = plt.subplots(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1,
                xticklabels=["Cat", "Dog"], yticklabels=["Cat", "Dog"])
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("True")
    ax1.set_title("Confusion Matrix")

    canvas1 = FigureCanvasTkAgg(fig1, master=dash)
    canvas1.get_tk_widget().pack()

    # Plot ROC curve
    fig2, ax2 = plt.subplots(figsize=(5, 4))
    ax2.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    ax2.plot([0, 1], [0, 1], linestyle="--")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.set_title("ROC Curve")
    ax2.legend()

    canvas2 = FigureCanvasTkAgg(fig2, master=dash)
    canvas2.get_tk_widget().pack()

    dash.mainloop()
def upload_image():
    path = filedialog.askopenfilename(
        filetypes=[("Images", "*.jpg *.jpeg *.png")]
    )
    if not path:
        return

    status_label.config(text="Analyzing image…")
    top.update_idletasks()

    pred_text, confidence, cam_path = predict_and_generate_cam(path)

    prediction_label.config(
        text=pred_text,
        fg="#22c55e" if "Cat" in pred_text else "#f97316"
    )

    confidence_bar["value"] = confidence * 100

    cam_img = Image.open(cam_path)
    cam_img.thumbnail((600, 600))
    tk_img = ImageTk.PhotoImage(cam_img)

    image_label.configure(image=tk_img)
    image_label.image = tk_img

    status_label.config(
        text=f"Confidence: {confidence:.2%}"
    )

# UPLOAD BUTTON
upload_button = tk.Button(
    top,
    text="📸 Upload Image",
    command=upload_image,
    font=("Segoe UI", 14, "bold"),
    bg=ACCENT,
    fg="black",
    activebackground="#0ea5e9",
    relief=tk.FLAT,
    padx=20,
    pady=10,
    cursor="hand2"
)
upload_button.pack(pady=30)
evaluate_button = tk.Button(
    top,
    text="📊 Evaluate Model",
    command=evaluate_model,
    font=("Segoe UI", 14, "bold"),
    bg="#a78bfa",
    fg="black",
    activebackground="#c4b5fd",
    relief=tk.FLAT,
    padx=20,
    pady=10,
    cursor="hand2"
)
evaluate_button.pack(pady=10)
top.mainloop()

Found 627 images belonging to 2 classes.
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


C:\Users\Jonah\miniconda3\Lib\site-packages\keras\src\models\functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_1']
Received: inputs=Tensor(shape=(1, 128, 128, 3))
  warnings.warn(msg)


Found 627 images belonging to 2 classes.
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
